# Internal mutablity

## Cell

`Cell` in Rust is a type that allows you to mutate the value inside it. It is a type that is used to wrap a value and provide interior mutability. It is used when you need to mutate a value inside an immutable reference. 

You can either:
* get a copy of the value inside the `Cell` using the `get()` method
* set the value inside the `Cell` using the `set()` method

`Cell` requires for a type to implement the `Copy` trait. 

In [69]:
mod node_with_cell {
    use std::cell::Cell;

    #[derive(Debug)]
    struct Node<'a>{
        value: Cell<i32>,
        adjacent_nodes: Vec<&'a Node<'a>> 
    }

    impl<'a> Node<'a> {
        fn new(value: i32) -> Node<'a> {
            Node {
                value: Cell::new(value),
                adjacent_nodes: Vec::new()
            }
        }

        fn add_adjacent_node(&mut self, node: &'a Node) {
            self.adjacent_nodes.push(node);
        }
    }

    fn add_one(node: &Node) {
        let current_value = node.value.get();
        node.value.set(current_value + 1);
        for adjacent_node in &node.adjacent_nodes {
            add_one(&adjacent_node);
        }
    }
}

In [70]:
fn main() {
    use node_with_cell::*;

    let mut a = Node::new(1);

    let mut b = Node::new(4);
    b.add_adjacent_node(&a);

    let mut c = Node::new(10);
    c.add_adjacent_node(&a);

    add_one(&mut b);

    println!("a: {:?}", a);  
    println!("b: {:?}", b);  
    println!("c: {:?}", c);      
}

main();

a: Node { value: Cell { value: 2 }, adjacent_nodes: [] }
b: Node { value: Cell { value: 5 }, adjacent_nodes: [Node { value: Cell { value: 2 }, adjacent_nodes: [] }] }
c: Node { value: Cell { value: 10 }, adjacent_nodes: [Node { value: Cell { value: 2 }, adjacent_nodes: [] }] }


## RefCell

If a type is not copyable you can use `RefCell` which is a type that allows you to mutate the value inside it.

`RefCell` allows to get:
* a mutable reference to the value inside it using the `borrow_mut()` method (this reference must be exclusive - it panics if there is already a reference to the value)
* an immutable reference to the value inside it using the `borrow()` method (this method panics if there is already a mutable reference to the value)

This rules are checked at runtime.

In [77]:
mod node_with_ref_cell {
    use std::cell::RefCell;

    #[derive(Debug)]
    pub struct Node<'a>{
        value: RefCell<String>,
        adjacent_nodes: Vec<&'a Node<'a>> 
    }

    impl<'a> Node<'a> {
        pub fn new(value: String) -> Node<'a> {
            Node {
                value: RefCell::new(value),
                adjacent_nodes: Vec::new()
            }
        }

        pub fn add_adjacent_node(&mut self, node: &'a Node) {
            self.adjacent_nodes.push(node);
        }
    }

    pub fn add_exclamation(node: &Node) {
        let mut current_value = node.value.borrow_mut();
        current_value.push_str("!");

        for adjacent_node in &node.adjacent_nodes {
            add_exclamation(&adjacent_node);
        }
    }
}

In [79]:
fn main() {
    use node_with_ref_cell::{Node, add_exclamation};

    let mut a = Node::new("Hello".to_string());

    let mut b = Node::new("World".to_string());
    b.add_adjacent_node(&a);

    let mut c = Node::new("Rust".to_string());
    c.add_adjacent_node(&a);

    add_exclamation(&mut b);

    println!("a: {:?}", a);  
    println!("b: {:?}", b);  
    println!("c: {:?}", c);      
}

main();

a: Node { value: RefCell { value: "Hello!" }, adjacent_nodes: [] }
b: Node { value: RefCell { value: "World!" }, adjacent_nodes: [Node { value: RefCell { value: "Hello!" }, adjacent_nodes: [] }] }
c: Node { value: RefCell { value: "Rust" }, adjacent_nodes: [Node { value: RefCell { value: "Hello!" }, adjacent_nodes: [] }] }


## RWLock

`RWLock` is a type that allows you to mutate the value inside it in thread-safe manner. It allows to get:

* a mutable reference to the value inside it using the `write()` method (this reference must be exclusive - it panics if there is already a reference to the value)
* an immutable reference to the value inside it using the `read()` method

In [100]:
mod node_with_rw_lock {
    use std::sync::{RwLock, Arc};

    #[derive(Debug)]
    pub struct Node {
        pub value: RwLock<String>,
        pub adjacent_nodes: Vec<Arc<Node>> 
    }

    impl Node {
        pub fn new(value: String) -> Node {
            Node {
                value: RwLock::new(value),
                adjacent_nodes: Vec::new()
            }
        }

        pub fn add_adjacent_node(&mut self, node: Arc<Node>) {
            self.adjacent_nodes.push(node);
        }
    }

    pub fn add_exclamation(node: &Node) {
        let mut current_value = node.value.write().unwrap();
        current_value.push_str("!");

        for adjacent_node in &node.adjacent_nodes {
            add_exclamation(&adjacent_node);
        }
    }
}

In [101]:
fn main() {
    use std::sync::{Arc, RwLock};

    use node_with_rw_lock::{Node, add_exclamation};

    let mut a = Arc::new(Node{value: RwLock::new("Hello".to_string()), adjacent_nodes: Vec::new()});

    let mut b = Arc::new(Node{value: RwLock::new("World".to_string()), adjacent_nodes: vec![a.clone()]});

    let mut c = Arc::new(Node{value: RwLock::new("Rust".to_string()), adjacent_nodes: vec![a.clone()]});

    let b_pinned = b.clone();
    let thd_1 = std::thread::spawn(move || {
        add_exclamation(&b_pinned);
    });
    
    let c_pinned = c.clone();
    let thd_2 = std::thread::spawn(move || {
        add_exclamation(&c_pinned);
    });

    thd_1.join().unwrap();
    thd_2.join().unwrap();

    println!("a: {:?}", a);  
    println!("b: {:?}", b);  
    println!("c: {:?}", c);      
}

main();

a: Node { value: RwLock { data: "Hello!!", poisoned: false, .. }, adjacent_nodes: [] }
b: Node { value: RwLock { data: "World!", poisoned: false, .. }, adjacent_nodes: [Node { value: RwLock { data: "Hello!!", poisoned: false, .. }, adjacent_nodes: [] }] }
c: Node { value: RwLock { data: "Rust!", poisoned: false, .. }, adjacent_nodes: [Node { value: RwLock { data: "Hello!!", poisoned: false, .. }, adjacent_nodes: [] }] }
